In [58]:
import numpy as np
from main import *
from copy import deepcopy
from random import choice

In [59]:
info = read_file("../Ref/CARP/CARP_samples/egl-e1-A.dat")
sp = shortest_path(info["graph"])
sp, info["tasks"]

(array([[  0,  32,  46, ..., 271, 226, 261],
        [ 32,   0,  14, ..., 239, 194, 229],
        [ 46,  14,   0, ..., 253, 208, 243],
        ...,
        [271, 239, 253, ...,   0, 119, 154],
        [226, 194, 208, ..., 119,   0,  35],
        [261, 229, 243, ..., 154,  35,   0]]),
 [((0, 1), 32),
  ((1, 0), 32),
  ((1, 2), 14),
  ((2, 1), 14),
  ((1, 3), 17),
  ((3, 1), 17),
  ((3, 4), 56),
  ((4, 3), 56),
  ((8, 9), 20),
  ((9, 8), 20),
  ((10, 11), 32),
  ((11, 10), 32),
  ((11, 15), 29),
  ((15, 11), 29),
  ((12, 15), 13),
  ((15, 12), 13),
  ((12, 13), 7),
  ((13, 12), 7),
  ((14, 16), 26),
  ((16, 14), 26),
  ((14, 17), 38),
  ((17, 14), 38),
  ((17, 18), 41),
  ((18, 17), 41),
  ((18, 19), 32),
  ((19, 18), 32),
  ((18, 20), 38),
  ((20, 18), 38),
  ((20, 21), 17),
  ((21, 20), 17),
  ((21, 74), 24),
  ((74, 21), 24),
  ((22, 30), 40),
  ((30, 22), 40),
  ((30, 31), 58),
  ((31, 30), 58),
  ((31, 32), 30),
  ((32, 31), 30),
  ((31, 33), 43),
  ((33, 31), 43),
  ((31, 34), 86),

In [60]:
depot, capacity = info["depot"], info["capacity"]
depot, capacity

(0, 305)

In [61]:
def rule1(sp, depot, tasks):
    """Rule 1: maximize the distance from the head of task to the depot"""
    func = lambda x: min(sp[depot, x[0][0]], sp[depot, x[0][1]])

In [62]:
def remove_task(tasks, task):
    tasks.remove(task)
    tasks.remove(((task[0][1], task[0][0]), task[1]))

In [65]:

remain_cap = capacity

route = [(0, 0)]
cur_vertex = 0  # 车正在哪个顶点
tasks = deepcopy(info["tasks"])
while True:
    valid_tasks = []
    for task in tasks:
        if task[1] <= remain_cap:  # 这个任务比剩余容量小，放得下
            valid_tasks.append(task)
    
    
    if len(valid_tasks) == 0:
        # 当没有有效的任务时，连回depot
        route.append((0, 0))
        break

    elif len(valid_tasks) == 2:
        # 只有一个有效任务，最短路径过去
        task = valid_tasks[0] if valid_tasks[0][0][0] < valid_tasks[1][0][0] else valid_tasks[1]
        pos = task[0]
        route.append(pos)
        cur_vertex = pos[0]  # 修改车的位置
        
        # 移除task
        remove_task(tasks, task)
        # 减少容量
        remain_cap -= valid_tasks[0][1]

        # 一定是最后一个了
        route.append((0, 0))
        break
    
    else:
        # 有多个任务满足条件，选一个距离最短的
        min_cost = min(valid_tasks, key=lambda x: sp[cur_vertex, x[0][0]])[1]

        # 检查是否有多个距离最短的点
        min_cost_tasks = [t for t in valid_tasks if t[1] == min_cost]
        # print(min_cost_tasks)
        if len(min_cost_tasks) == 1:  # 只有一个
            task = min_cost_tasks[0]
        else:  # 多个
            # TODO 使用五种Rules
            task = choice(min_cost_tasks)
        
        pos = task[0]
        route.append(pos)
        cur_vertex = pos[0]
        
        # 移除task
        remove_task(tasks, task)
        remain_cap -= min_cost

route

[(0, 0),
 (68, 57),
 (68, 58),
 (68, 3),
 (58, 57),
 (57, 56),
 (45, 46),
 (15, 12),
 (15, 11),
 (13, 12),
 (50, 20),
 (0, 0)]